In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyserini==0.9.4.0

In [ ]:
!pip3 install transformers
!pip3 install unidecode

In [ ]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import join
import unidecode
import re
import logging
from tqdm.notebook import tnrange
import glob
import json

#For ploting results
import matplotlib.pyplot as plt

# DL Libraries
from transformers import BertModel, AdamW, BertTokenizer,RobertaTokenizer, BertConfig, get_linear_schedule_with_warmup,RobertaModel, RobertaForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,TensorDataset)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from statistics import mean 
#NLTK Libraries
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=6b6fefa8420998974efd213656d52f47304c5b5fde51e67f9ed14a6b654f7feb
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.5 GB  |     Proc size: 544.2 MB
GPU RAM Free: 15069MB | Used: 10MB | Util   0% | Total     15079MB


In [ ]:
from pyserini.search import SimpleSearcher
import pandas as pd
from os.path import join
import numpy as np

# searcher = SimpleSearcher('/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini/pyserini/indexes/lucene-index-qasc')
searcher = SimpleSearcher('/content/drive/MyDrive/Courses/NLP/Project/data/HOTPOTQA_Corpus/anserini/pyserini/indexes/lucene-index-hotpotqa')

# /content/drive/MyDrive/Courses/NLP/Project/data/HOTPOTQA_Corpus/anserini/pyserini/indexes/lucene-index-hotpotqa

searcher.set_bm25(1.2, 0.75)

hits = searcher.search("The American")

# Print the first 10 hits:
for i in range (0,len(hits)):
    print(f'{hits[i].docid:15} {hits[i].score:.5f}')

12210           1.25990
104021          1.21460
77994           1.20960
89711           1.20960
95879           1.20960
2037            1.20210
47974           1.18920
29845           1.18660
41591           1.18660
95881           1.18660


In [ ]:
for i in range (0,len(hits)):
  doc = searcher.doc(str(hits[i].docid))
  print(doc.raw())

"Italian Americans (Italian: ""italoamericani ""or"" italo-americani"" ] ) are an ethnic group consisting of Americans who have full or partial ancestry from Italy. Italian Americans are the fourth largest European ethnic group in the United States (not including American ethnicity, an ethnonym used by many in the United States; overall, Italian Americans rank seventh, behind German American, Irish American, African American, English American, American, and Mexican American).."
Indian Americans or Indo-Americans are Americans whose ancestry belongs to any of the many ethnic groups of the Republic of India. As the most socio-economically successful minority ethnic group in the U.S., Indian Americans comprise 4 million people, representing around 1% of the U.S. population as of 2015. Indian Americans are the country's third-largest Asian group alone or in combination with other races after Chinese Americans and Filipino Americans, according to 2015 American Community Survey data. The U.S

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("device: {} n_gpu: {}".format(device, n_gpu))

device: cuda n_gpu: 1


# Data Loader

In [ ]:
def create_dataloader(tokenizer, df):
    input_ids= list()
    attention_masks= list()

    special_sentences_1 = [sentence for i, sentence in enumerate(df.question)]
    special_sentences_2 = [" [SEP] " + str(sentence) for i, sentence in enumerate(df.answer)]
    special_sentences = [i + j for i, j in zip(special_sentences_1, special_sentences_2)]

    for sentence in special_sentences:
      encoded_text = tokenizer.encode_plus(sentence, max_length=512, add_special_tokens=True, return_token_type_ids=False, 
                                       padding='max_length', return_attention_mask=True, truncation=True)
      input_ids.append(encoded_text['input_ids'])
      attention_masks.append(encoded_text['attention_mask'])

    inputs = torch.tensor(input_ids).to(device)
    masks = torch.tensor(attention_masks).to(device)
    # gold_labels = torch.tensor(df.label.tolist()).to(device)
  
    data = TensorDataset(inputs, masks)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=1)

    return dataloader

## Evaluation

In [ ]:
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

In [ ]:
def recall_at_k(actual, predicted, k):
  count=0
  predicted= predicted[0:k]
  for a in actual:
    if a in predicted:
      count+=1
  result = count/len(actual)
  # print("count=",count,"length",len(actual),"result", result)
  return count/len(actual)

In [ ]:
def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

In [ ]:
def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

#Fetch sentences from corpus

In [ ]:
def get_candidate_passages(query):

  candidate_passages=[]
  hits = searcher.search(query, k=50)
  # return the first top 10 hits:
  for hit in hits:
    doc = searcher.doc(str(hit.docid))
    candidate_passages.append(doc.raw().replace('"', ''))
  
  return candidate_passages

#Model

In [ ]:
#Class for Regression
class Regressor(nn.Module):

  def __init__(self, model_path):
    super(Regressor, self).__init__()
    self.bert = RobertaModel.from_pretrained(model_path)
    self.out = nn.Linear(self.bert.config.hidden_size, 1)

  def forward(self, input_ids, attention_mask):
    output, pooler_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    score= self.out(pooler_out)
    return score

In [ ]:
#classification model

# model_path= '/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/sia_experiment/model_QASCrobertabase_40ktraindataset_10epochs'
model_path= '/content/drive/MyDrive/Courses/NLP/Project/models/sia_trained_roberta_model'

#Load Model
model= Regressor(model_path)
lr_weights= torch.load(join(model_path, 'model_state.bin'))
model.out.load_state_dict(lr_weights)
model.to(device)


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Load Dev set


In [ ]:
def generate_supfacts_list(df, start,end):
    new_df = pd.DataFrame(columns=['question','answer'])   

    for i in tnrange(start,end):
      
      cntxt_len = len(df['context'][i])
      supporting_facts = df['supporting_facts'][i]
      question =  df['question'][i]
      fact_list = []
      for fact in  supporting_facts:
        fact_title = fact[0]
        for j  in range(0, cntxt_len):
          passage_title =  df['context'][i][j][0]
          if fact_title == passage_title:
            passage  = df['context'][i][j][1]
            text  = "".join(passage)
            fact_list.append(text)
              
      new_row = { 'question':question,'answer': fact_list }
      new_df =  new_df.append(new_row,ignore_index= True)

    return  new_df

In [ ]:

start = 0
end = 7405
FOLDER = '/content/drive/My Drive/Courses/NLP/Project/data'
JSON_FILE =  FOLDER +"/hotpot_dev_distractor_v1.json"
df = pd.read_json(JSON_FILE)
data_df = generate_supfacts_list(df, start,end)
data_df.to_csv('/content/drive/MyDrive/Courses/NLP/Project/data/dev_facts_0_7405.csv', header=True, index=False)

In [ ]:
len(data_df["answer"][0])

2

#evaluation

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_combined_fact=0
for index, row in data_df.iterrows():
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      score = model(ip_ids, attention_mask=masks)
      pos_prob.append(score.cpu().detach().numpy())
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()
  org_ans = row.answer
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  print(index)
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
print("Precision @5: ",mean(precision_at_5))
print("Precision @10: ",mean(precision_at_10))
print("Recall @5: ",mean(recall_at_5))
print("Recall @10: ",mean(recall_at_10))
print("Recall @20: ",mean(recall_at_20))
print("Mean Average Precision: ",mean_average_precision(rs))
print("Mean Reciprocal Rank: ",mean_reciprocal_rank(rs))
print("contains_combined_fact",contains_combined_fact)


In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact2=0
for index, row in data_df.iterrows():
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      score = model(ip_ids, attention_mask=masks)
      pos_prob.append(score.cpu().detach().numpy())
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  org_ans=[]
  org_ans.append(row.fact_2)

  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_2:
      contains_fact2+=1
  # if(index==1000):
  #   break
  print(index)
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
print("QASC DEV experiment 4 with considering only fact2")
print("Precision @5: ",mean(precision_at_5))
print("Precision @10: ",mean(precision_at_10))
print("Recall @5: ",mean(recall_at_5))
print("Recall @10: ",mean(recall_at_10))
print("Recall @20: ",mean(recall_at_20))
print("Mean Average Precision: ",mean_average_precision(rs))
print("Mean Reciprocal Rank: ",mean_reciprocal_rank(rs))
print("contains_fact2",contains_fact2)

## **Classification Baseline Model experimentation**

---



# Loading the model

In [ ]:
#classification model

model_path= '/content/drive/MyDrive/NLP/Project/models/sia_trained_roberta_model'
model= RobertaForSequenceClassification.from_pretrained(model_path)
lr_weights= torch.load(join(model_path, 'model_state.bin'))
#model.out.load_state_dict(lr_weights)
model.to(device)



tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Some weights of the model checkpoint at /content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/sia_experiment/roberta_cls_model/ were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/sia_experiment/roberta_cls_model/ and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_combined_fact=0
for index, row in data_df.iterrows():
  print(index)
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      logits = model(ip_ids, attention_mask=masks)
      logits= logits[0].squeeze(0)
      pred_logits = logits.cpu().detach().numpy()
      pos_prob.append(pred_logits[1])
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  org_ans.append(row.combined_fact)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.combined_fact:
      contains_combined_fact+=1
  
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("combined fact:",contains_combined_fact)

In [ ]:
print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("combined fact:",contains_combined_fact)

0.0023758099352051837
0.003455723542116631
0.011879049676025918
0.032397408207343416
0.12419006479481641
0.014276984723627631
0.013826532359672
combined fact: 127


In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_all_three=0
contains12=0
contains1c=0
contains2c=0
for index, row in data_df.iterrows():
  print(index)
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      logits = model(ip_ids, attention_mask=masks)
      logits= logits[0].squeeze(0)
      pred_logits = logits.cpu().detach().numpy()
      pos_prob.append(pred_logits[1])
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  org_ans=[]
  org_ans.append(row.combined_fact)
  org_ans.append(row.fact_1)
  org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  if ((row.fact_1 in top_k_ans)and (row.fact_2 in top_k_ans) and (row.combined_fact in top_k_ans)):
    contains_all_three+=1
  if ((row.fact_1 in top_k_ans)and (row.fact_2 in top_k_ans) and (row.combined_fact not in top_k_ans)):
    contains12+=1
  if ((row.fact_1 in top_k_ans) and (row.combined_fact in top_k_ans) and (row.fact_2 not in top_k_ans)):
    contains1c+=1
  if ((row.fact_2 in top_k_ans) and (row.combined_fact in top_k_ans)and (row.fact_1 not in top_k_ans)):
    contains2c+=1

  
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
  

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_all_three",contains_all_three)
print("contains_fact1_and_fact2",contains12)
print("contains_fact1_and_factc",contains1c)
print("contains_fact2_and_factc",contains2c)

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact1=0

for index, row in data_df.iterrows():
  print(index)
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      logits = model(ip_ids, attention_mask=masks)
      logits= logits[0].squeeze(0)
      pred_logits = logits.cpu().detach().numpy()
      pos_prob.append(pred_logits[1])
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  # org_ans.append(row.combined_fact)
  org_ans.append(row.fact_1)
  # org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_1:
      contains_fact1+=1
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_fact1",contains_fact1)

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_fact2=0

for index, row in data_df.iterrows():
  print(index)
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question
  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      logits = model(ip_ids, attention_mask=masks)
      logits= logits[0].squeeze(0)
      pred_logits = logits.cpu().detach().numpy()
      pos_prob.append(pred_logits[1])
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  # org_ans.append(row.combined_fact)
  # org_ans.append(row.fact_1)
  org_ans.append(row.fact_2)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.fact_2:
      contains_fact2+=1
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
print("contains_fact2_and_factc",contains_fact2)

# **BM25 Baseline Model Result Evaluation**



In [ ]:
def get_candidate_passages(query):

  candidate_passages=[]
  scores=[]
  hits = searcher.search(query, k=50)
  # return the first top 10 hits:
  for hit in hits:
    doc = searcher.doc(str(hit.docid))
    candidate_passages.append(doc.raw().replace('"', ''))
    scores.append(hit.score)
  
  return candidate_passages,scores

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_combined_fact=0
for index, row in data_df.iterrows():
  # if index  >   2000:
  #   break;
  print(index)
  answers,score= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(list(zip(answers,score)), columns=['answer','score'])
  temp_df['question']=row.question
  # dataloader= create_dataloader(tokenizer, temp_df)
  # pos_prob= []
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()

  # doc= doc_passages[str(row.DocumentID)]
  # passages_no= row.RelevantPassages.split(',')
  # org_ans=[]
  # for no in passages_no:
  #   org_ans.append(doc[str(no)])
  # org_ans.append(row.combined_fact)
  org_ans = row.answer
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  # for ans in top_k_ans:
  #   if ans==row.combined_fact:
  #     contains_combined_fact+=1
  
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print(mean(precision_at_5))
print(mean(precision_at_10))
print(mean(recall_at_5))
print(mean(recall_at_10))
print(mean(recall_at_20))
print(mean_average_precision(rs))
print(mean_reciprocal_rank(rs))
# print("combined fact:",contains_combined_fact)